# Time series


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import timedelta

import numpy as np
import pandas as pd

## Use case - multiple time series signals

### Input: **datetime indexed time(zone-aware) series dataframes**

[documentation datetime index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)

In [3]:
# load the data (and add a random offset to it), as multimodal data might be
# with some small offsets
df_gsr = pd.read_feather("data/gsr.feather")
df_gsr["timestamp"] += timedelta(milliseconds=np.random.randint(-125, 125))
df_gsr.set_index("timestamp", inplace=True)

df_tmp = pd.read_feather("data/tmp.feather")
df_tmp["timestamp"] += timedelta(milliseconds=np.random.randint(-125, 125))
df_tmp.set_index("timestamp", inplace=True)

In [4]:
type(df_gsr.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [5]:
print(df_gsr.shape)
df_gsr.head(2)

(31998, 1)


,EDA
timestamp,
2017-06-13 10:33:40.061000+02:00,0.000000
2017-06-13 10:33:40.311000+02:00,0.107451


In [6]:
print(df_tmp.shape)
df_tmp.head(2)

(31992, 1)


,TMP
timestamp,
2017-06-13 10:33:39.982000+02:00,382.21
2017-06-13 10:33:40.232000+02:00,382.21


---

### Intermezzo Joining multiple time series: `pd.merge_asof`

[documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html)

In [7]:
# the name of the index columns is "timestamp"
df_tot = pd.merge_asof(
    df_tmp[4:], df_gsr, on="timestamp", direction="nearest"
).set_index("timestamp")
df_tot.head(2)

,TMP,EDA
timestamp,,
2017-06-13 10:33:40.982000+02:00,31.15,0.153501
2017-06-13 10:33:41.232000+02:00,31.15,0.148384


In [8]:
# preserve causality -> direction = backward
#  A “backward” search selects the last row in the right DataFrame whose ‘on’
#  key is less than or equal to the left’s key.
df_tot = pd.merge_asof(
    df_tmp[10:], df_gsr, on="timestamp", direction="backward"
).set_index("timestamp")

certainly look at `tolerance` and `direction` argument of this function!

---

# Classical machine learning -> feature extraction

Most common way to extract time series features: a **window-strided** manner

challenges:
* Always assumes fixed window size & stride
* No efficient implementations for dataframes (pd.rolling assumes same input<->output dimensions --> no stride possible) 
* No support aggregation multiple time series 

--> Writing my own code 

In [9]:
import sys

# time series feature extraction
sys.path.append("time_series/")
# Serialization
import dill as pickle
import scipy.stats as ss

from time_series import FeatureCollection, NumpyFuncWrapper
from time_series.features import FeatureDescriptor

# todo add example for MultipleFeatureDescriptors class
# from time_series.features import MultipleFeatureDescriptors

pickle.settings["recurse"] = True  # allows to serialize lambda's YAY!

## Defining functions

In [10]:
# --------------------- some custom feature extraction functions ---------------------
# 1. one-to-many functions (as quantiles sort the windowed data) you might
#    want to calculate them in 1 step
quantiles = [0.25, 0.5, 0.75]
f_quantiles = NumpyFuncWrapper(
    np.quantile, output_names=[f"quantile_{q}" for q in quantiles], q=quantiles
)  # fyi: you can pass kwargs (in this case q=quantiles)


# 2. in-line functions
def slope(x):
    return np.polyfit(np.arange(0, len(x)), x, 1)[0]


def rms(x):
    return np.sqrt(np.mean(x ** 2))


f_slope = NumpyFuncWrapper(slope, output_names="slope")
f_rms = NumpyFuncWrapper(rms, output_names="rms")
f_area = NumpyFuncWrapper(np.sum, output_names="area")

segment_funcs = [
    np.mean,
    np.std,
    np.var,
    np.max,
    np.min,
    ss.skew,
    ss.kurtosis,
    f_quantiles,
    f_slope,
    f_rms,
    f_area,
]

## Lambdas

Lambda's aren't serializable by default, making them hard to use in other enivronments (such als multiprocessing pools).

Hence, the `NumpyFuncWrapper` class will output a warning if lambda's are being used.

**note**:
* A possible workaround to still use lambdas and multiprocessing is using [dill](https://github.com/uqfoundation/dill) (for serialization) and [pathos](https://github.com/uqfoundation/pathos) (for multiprocessing).

In [11]:
_ = NumpyFuncWrapper(lambda x: np.mean(x), output_names="lambda_mean")

/home/jonas/git/gIDLab/predict/time_series/time_series/features/function_wrapper.py:38: UserWarning: 
Function: lambda_mean is a lambda, thus not pickle-able.
This might give problems with multiprocessing.
  f"\nFunction: {output_names} is a lambda, thus not pickle-able.\n"


## Use case 1: single feature extraction for temperature signal

### Fixed window size

**TMP**

In [12]:
fs_tmp = 4
tmp_win_size = 60 * fs_tmp
tmp_stride_size = 30 * fs_tmp
tmp_feat_extr = FeatureCollection(
    [
        FeatureDescriptor(
            key="TMP", window=tmp_win_size, stride=tmp_stride_size, function=f
        )
        for f in segment_funcs
    ]
)
tmp_feat_extr

FeatureCollection(
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
	FeatureDescriptor(TMP, 240, 120) 
)

In [13]:
%%time
tmp_feat_extr.calculate(df_tmp, merge_dfs=True).sample(2)

CPU times: user 50.4 ms, sys: 43.1 ms, total: 93.6 ms
Wall time: 278 ms


,TMP_mean__w=240_s=120,TMP_std__w=240_s=120,TMP_var__w=240_s=120,TMP_amax__w=240_s=120,TMP_amin__w=240_s=120,TMP_skew__w=240_s=120,TMP_kurtosis__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,TMP_rms__w=240_s=120,TMP_area__w=240_s=120
timestamp,,,,,,,,,,,,,
2017-06-13 12:46:09.732000+02:00,31.390,0.017127,0.000293,31.43,31.35,-4.496006e-17,-0.892562,31.37,31.39,31.41,0.000028,31.390005,7533.60
2017-06-13 11:04:39.732000+02:00,32.873,0.032470,0.001054,32.93,32.81,-4.035668e-01,-1.126057,32.84,32.89,32.89,0.000413,32.873016,7889.52


In [14]:
%%time
tmp_feat_extr.calculate(df_tmp, merge_dfs=True, njobs=1).sample(2)

CPU times: user 23.8 ms, sys: 3.85 ms, total: 27.6 ms
Wall time: 220 ms


,TMP_mean__w=240_s=120,TMP_std__w=240_s=120,TMP_var__w=240_s=120,TMP_amax__w=240_s=120,TMP_amin__w=240_s=120,TMP_skew__w=240_s=120,TMP_kurtosis__w=240_s=120,TMP_quantile_0.25__w=240_s=120,TMP_quantile_0.5__w=240_s=120,TMP_quantile_0.75__w=240_s=120,TMP_slope__w=240_s=120,TMP_rms__w=240_s=120,TMP_area__w=240_s=120
timestamp,,,,,,,,,,,,,
2017-06-13 11:02:39.732000+02:00,32.519500,0.039048,0.001525,32.59,32.47,0.714449,-0.902118,32.49,32.50,32.55,0.000509,32.519523,7804.68
2017-06-13 11:34:09.732000+02:00,32.749333,0.011528,0.000133,32.77,32.73,-0.000387,-0.000067,32.75,32.75,32.75,-0.000052,32.749335,7859.84


### Arbitrary window size

**GSR**

In [15]:
# PoC: we will select a random combination of the window_size stride combination
window_size_s = [60, 120]
stride_size_s = [30, 10, 20]
fs_gsr = 4

import random

gsr_feat_extr = FeatureCollection(
    [
        FeatureDescriptor(
            key="EDA",
            window=random.choice(window_size_s) * fs_tmp,
            stride=random.choice(stride_size_s) * fs_tmp,
            function=f,
        )
        for f in segment_funcs
    ]
)
gsr_feat_extr

FeatureCollection(
	FeatureDescriptor(EDA, 480, 120) 
	FeatureDescriptor(EDA, 240, 120) 
	FeatureDescriptor(EDA, 240, 120) 
	FeatureDescriptor(EDA, 480, 80) 
	FeatureDescriptor(EDA, 240, 120) 
	FeatureDescriptor(EDA, 480, 40) 
	FeatureDescriptor(EDA, 480, 40) 
	FeatureDescriptor(EDA, 480, 40) 
	FeatureDescriptor(EDA, 240, 40) 
	FeatureDescriptor(EDA, 240, 40) 
	FeatureDescriptor(EDA, 240, 120) 
)

In [16]:
%%time
gsr_feat_extr.calculate(df_gsr, merge_dfs=True).sample(2)
# gsr_feat_extr.get_results(merge_dfs=True)

CPU times: user 63.8 ms, sys: 60.5 ms, total: 124 ms
Wall time: 384 ms


,EDA_mean__w=480_s=120,EDA_std__w=240_s=120,EDA_var__w=240_s=120,EDA_amin__w=240_s=120,EDA_area__w=240_s=120,EDA_amax__w=480_s=80,EDA_skew__w=480_s=40,EDA_kurtosis__w=480_s=40,EDA_quantile_0.25__w=480_s=40,EDA_quantile_0.5__w=480_s=40,EDA_quantile_0.75__w=480_s=40,EDA_slope__w=240_s=40,EDA_rms__w=240_s=40
timestamp,,,,,,,,,,,,,
2017-06-13 12:15:19.811000+02:00,NaN,NaN,NaN,NaN,NaN,0.882966,-0.215916,-1.324982,0.629689,0.739058,0.798540,-0.000751,0.639235
2017-06-13 12:43:59.811000+02:00,NaN,NaN,NaN,NaN,NaN,0.162722,0.025688,-1.514857,0.136818,0.143534,0.155047,0.000061,0.154009


### Multiple time series

In [17]:
# Construct the feature FeatureCollection
#   =  higher order wrapper which aggregates the featuredescriptions
feature_extraction = FeatureCollection(feature_desc_list=[gsr_feat_extr, tmp_feat_extr])

In [18]:
df_feat = feature_extraction.calculate([df_gsr, df_tmp], merge_dfs=True)

# LAYD: Look At Your Data

In [19]:
import ipywidgets as widgets
import plotly.graph_objects as go
from ipywidgets import interact_manual
from plotly.subplots import make_subplots

In [20]:
df_dict = {"tmp": df_tmp, "gsr": df_gsr}

In [21]:
feat_widget = widgets.SelectMultiple(options=df_feat.columns)
sig_widget = widgets.SelectMultiple(options=["gsr", "tmp"])

In [22]:
@interact_manual
def visuzalize(features=feat_widget, signals=sig_widget):
    row_titles = list(signals) + ["features"] if len(features) else []
    fig = make_subplots(
        rows=len(row_titles),
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1 / len(row_titles),
        row_titles=row_titles,
    )
    fig.update_layout(height=300 * len(row_titles))

    # first, visualize the "raw" signals
    row_idx = 1
    for sig in signals:
        df_sig = df_dict[sig][10:].resample("1s").mean()
        for col in set(df_sig.columns).difference(["index", "timestamp"]):
            fig.add_trace(
                go.Scattergl(x=df_sig.index, y=df_sig[col], name=col, hoverinfo="skip"),
                row=row_idx,
                col=1,
            )
        row_idx += 1

    # then visualize the features
    df_f = df_feat[3:]
    for feature in features:
        fig.add_trace(
            go.Scattergl(
                x=df_f.index,
                y=df_f[feature],
                name=feature,
                hoverinfo="skip",
                showlegend=True,
            ),
            row=row_idx,
            col=1,
        )

    return fig

interactive(children=(SelectMultiple(description='features', options=('EDA_mean__w=480_s=120', 'EDA_std__w=240…